In [ ]:
candidate_page_name = ["DoctorKoWJ", "YaoTurningTaipei", "tingshouchung", "professorofpower"]

In [ ]:
import requests
import time
import json


class FbPostCrawler:
    def __init__(self, page_name, since=None, until=None):
        self.access_token = "&access_token=" + "EAACEdEose0cBABwwmb8zl7alxwGVyzqYEl6loWPocUsHwZBHViGZB1BYZB4p0ZBF9KF9dnZA8ZBj1zxQHAZBXo6yuxpJ3BnViMRnTMKSZCVddkYpfkwQsQibVCZAtTiZAABbWRSW2esC2cKoyURID1lXZAu8IsU4JtvP3NLozrq94DqdsfVPvm64rE8EplTEes09AnotUakunvmJgZDZD"
        self.graph_url = "https://graph.facebook.com/v3.0/"
        self.page_url = self.graph_url + "%s/posts?" %page_name
        self.since, self.until = since, until
        try:
            self.file_name = page_name+"_post_"+self.since+"_"+self.until+".txt"    #讀寫檔用
        except:
            self.file_name = "post_all.txt"
        self.all_posts = list()    #紀錄所有貼文
        
    def getPosts(self, next_posts=None):
#         print("alive")
        time_range = "%s%s" %(""if self.since is None else "&since="+self.since, ""if self.until is None else "&until="+self.until)    #設定起訖時間
        post_url = (self.page_url+ self.access_token + time_range) if next_posts is None else next_posts    #遞迴用url
        
        posts_json = json.loads(requests.get(post_url).text)
        for data in posts_json["data"]:
            self.all_posts.append(data)
            
        if ((next_posts is not None) or ((self.since is not None) and (self.until is not None))):
            try:
                self.getPosts(posts_json["paging"]["next"])    #遞迴直到沒有next
            except:
                print("get post done")
        
    def printPosts(self):
        if(len(self.all_posts) == 0):
            print("empty")
            return
        print("count: " + str(len(self.all_posts)))
        for element in self.all_posts:
            print(element["created_time"])
    
    def writePosts(self):
            with open(self.file_name, "w") as writer:
                writer.write(json.dumps(self.all_posts))
            print("write post done")
    
    def readPosts(self, other_file=None):
        with open(self.file_name if other_file is None else other_file, "r") as reader:
            self.all_posts = json.load(reader)
        print("read post done")
        
#######################################################
    def getComments(self, next_comments= None):
        for post in self.all_posts:
            all_comments_list = list()
            
            def _get_all_comments(next_url = None):
                comments_url = self.graph_url + post["id"] + "/comments?" + self.access_token if next_url is None else next_url    #遞迴用url
                comments_json = json.loads(requests.get(comments_url).text)
                all_comments_list.extend(comments_json["data"])
                try:
                    get_all_comments(comments_json["paging"]["next"])
                except:
                    pass
                
            _get_all_comments()
            post["comments"] = all_comments_list
            
        print("get comments done")

In [ ]:
page_name = candidate_page_name[0]
posts_since = "2018-1-1"
posts_until = "2018-6-15"

post_crawler = FbPostCrawler(page_name, posts_since, posts_until)
# post_crawler.getPosts()
# post_crawler.getComments()
# crawler.printPosts()
# crawler.writePosts()
# crawler.readPosts()

In [ ]:
class FbReactionCrawler:
    def __init__(self, page_name, since=None, until=None):
        self.access_token = "&access_token=" + "EAACEdEose0cBABwwmb8zl7alxwGVyzqYEl6loWPocUsHwZBHViGZB1BYZB4p0ZBF9KF9dnZA8ZBj1zxQHAZBXo6yuxpJ3BnViMRnTMKSZCVddkYpfkwQsQibVCZAtTiZAABbWRSW2esC2cKoyURID1lXZAu8IsU4JtvP3NLozrq94DqdsfVPvm64rE8EplTEes09AnotUakunvmJgZDZD"
        self.graph_url = "https://graph.facebook.com/v3.0/"
        self.page_url = self.graph_url + "%s/posts?" %page_name
        self.since, self.until = since, until
        try:
            self.file_name = page_name+"_reaction_"+self.since+"_"+self.until+".txt"    #讀寫檔用
        except:
            self.file_name = page_name+"_reaction_all.txt"
        self.all_reactions = list()    #紀錄所有貼文
        
    def getReactions(self, next_posts=None):
        print("alive")
        time_range = "%s%s" %(""if self.since is None else "&since="+self.since, ""if self.until is None else "&until="+self.until)    #設定起訖時間
        post_url = (self.page_url+ self.access_token + time_range) if next_posts is None else next_posts    #遞迴用url
        posts_json = json.loads(requests.get(post_url).text)
        
        for data in posts_json["data"]:
            reaction_url = self.graph_url + data["id"] + "/?fields=reactions.type(NONE).limit(0).summary(1).as(ALL),reactions.type(LIKE).limit(0).summary(1).as(LIKE),reactions.type(LOVE).limit(0).summary(1).as(LOVE),reactions.type(HAHA).limit(0).summary(1).as(HAHA),reactions.type(WOW).limit(0).summary(1).as(WOW),reactions.type(SAD).limit(0).summary(1).as(SAD),reactions.type(ANGRY).limit(0).summary(1).as(ANGRY)" +self.access_token
            reaction_json = json.loads(requests.get(reaction_url).text)
            self.all_reactions.append({"created_time":data["created_time"], "id":data["id"],
                                        "reactions":[reaction_json["ALL"]["summary"]["total_count"],
                                                   reaction_json["LIKE"]["summary"]["total_count"],
                                                   reaction_json["LOVE"]["summary"]["total_count"],
                                                   reaction_json["HAHA"]["summary"]["total_count"],
                                                   reaction_json["WOW"]["summary"]["total_count"],
                                                   reaction_json["SAD"]["summary"]["total_count"],
                                                   reaction_json["ANGRY"]["summary"]["total_count"]]})
        if ((next_posts is not None) or ((self.since is not None) and (self.until is not None))):
            try:
                self.getReactions(posts_json["paging"]["next"])    #遞迴直到沒有next
            except:
                print("get reacition done")
    
    def writeReactions(self):
            with open(self.file_name, "w") as writer:
                writer.write(json.dumps(self.all_reactions))
            print("write reaction done")
    
    def readReactions(self, other_file=None):
        with open(self.file_name if other_file is None else other_file, "r") as reader:
            self.all_reactions = json.load(reader)
        print("read reaction done")

In [ ]:
page_name = candidate_page_name[0]
posts_since = "2018-1-1"
posts_until = "2018-6-15"

reaction_crawler = FbReactionCrawler(page_name, posts_since, posts_until)
# reaction_crawler.getReactions()
# reaction_crawler.writeReactions()
# reaction_crawler.readReactions()

reaction_rank = sorted(reaction_crawler.all_reactions, key=lambda k: k["reactions"][0], reverse=True)
for post in reaction_rank[:5]:
    print(post["created_time"][5:7]+post["created_time"][8:10] +","+ str(post["reactions"]))
    print("https://www.facebook.com/"+post["id"])

In [ ]:
class FbCountCrawler:
    def __init__(self, page_name, since=None, until=None):
        self.access_token = "&access_token=" + "EAACEdEose0cBABwwmb8zl7alxwGVyzqYEl6loWPocUsHwZBHViGZB1BYZB4p0ZBF9KF9dnZA8ZBj1zxQHAZBXo6yuxpJ3BnViMRnTMKSZCVddkYpfkwQsQibVCZAtTiZAABbWRSW2esC2cKoyURID1lXZAu8IsU4JtvP3NLozrq94DqdsfVPvm64rE8EplTEes09AnotUakunvmJgZDZD"
        self.graph_url = "https://graph.facebook.com/v3.0/"
        self.page_url = self.graph_url + "%s/posts?" %page_name
        self.since, self.until = since, until
        try:
            self.file_name = page_name+"_count_"+self.since+"_"+self.until+".txt"    #讀寫檔用
        except:
            self.file_name = page_name+"_count_all.txt"
        self.all_counts = list()    #紀錄所有貼文
        
    def getCounts(self, next_posts=None):
        print("alive")
        time_range = "%s%s" %(""if self.since is None else "&since="+self.since, ""if self.until is None else "&until="+self.until)    #設定起訖時間
        post_url = (self.page_url+ self.access_token + time_range) if next_posts is None else next_posts    #遞迴用url 
        posts_json = json.loads(requests.get(post_url).text)
        
        for data in posts_json["data"]:
            count_url = self.graph_url + data["id"] + "/?fields=shares,comments.limit(0).summary(1)" + self.access_token
            count_json = json.loads(requests.get(count_url).text)
            self.all_counts.append({"created_time":data["created_time"], "id":data["id"], "comments_count":count_json["comments"]["summary"]["total_count"], "shares_count":0 if "shares" not in count_json else count_json["shares"]["count"]})
            
        if ((next_posts is not None) or ((self.since is not None) and (self.until is not None))):
            try:
                self.getCounts(posts_json["paging"]["next"])    #遞迴直到沒有next
            except:
                print("get count done")
    
    def writeCounts(self):
            with open(self.file_name, "w") as writer:
                writer.write(json.dumps(self.all_counts))
            print("write count done")
    
    def readCounts(self, other_file=None):
        with open(self.file_name if other_file is None else other_file, "r") as reader:
            self.all_counts = json.load(reader)
        print("read count done")

In [ ]:
page_name = candidate_page_name[3]
posts_since = "2018-1-1"
posts_until = "2018-6-15"

count_crawler = FbCountCrawler(page_name, posts_since, posts_until)
count_crawler.getCounts()
count_crawler.writeCounts()
count_crawler.readCounts()

comment_rank = sorted(count_crawler.all_counts, key=lambda k: k["comments_count"], reverse=True)
share_rank = sorted(count_crawler.all_counts, key=lambda k: k["shares_count"], reverse=True)
print("comment:")
for post in comment_rank[:5]:
    print(post["created_time"][5:7]+post["created_time"][8:10] +","+ str(post["comments_count"]))
    print("https://www.facebook.com/"+post["id"])
print("\nshare:")
for post in share_rank[:5]:
    print(post["created_time"][5:7]+post["created_time"][8:10] +","+ str(post["shares_count"]))
    print("https://www.facebook.com/"+post["id"])